## Juntar tablas de diferentes años, seleccionar atributos más relevantes y generar promedios móviles de las Partículas PM

En este notebook se juntan los archivos de contaminación y meteorología de cada año en un solo dataframe. Adicionalmente vamos a generar columnas de información temporal para cada medición y por úlitmo se va a obtener el promedio móvil de el PM10 y PM2.5. 

__Pasos y descripción general del notebook__


1. Juntar los datos de las mediciones por año en un solo dataframe con el objetivo de agilizar su manipulación.


2. También se deshecharan las columnas "PBa","PMCO","UVA","UVB" y "WDR" debido a que éstos atributos no son compartidos en tiempo real por lo que no es posible hacer uso de éstas para el pronóstico. En el caso de "WDR" si se comparte en tiempo real sin embargo, se concluyó que este no aporta información a la descripción del fenómeno. 


3. Generar columnas con información temporal como la hora, día y mes en que se generó la medición, estos atributos son de ultilidad para utilizar los patrones temporales de temporalidad en el fenómeno de contaminación.


4. Se filtran los valores de PM10 y PM2.5 que salen del percentil .85. 


5. Por último generamos el atributo PM10mean y PM25mean que representarán el promedio móvil de las últimas 24 horas de los dos contaminantes.


- __Datos recibidos:__ Limpieza de datos inicial
- __Responsable:__ Daniel Bustillos
- __Contacto:__  juandaniel.bucam@gmail.com


In [2]:
import numpy as np
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None 

## 1. 
### Juntar los datos de las mediciones por año en un solo dataframe con el objetivo de agilizar su manipulación.

Declaramos los nombres de todos los años con los que se generarán los modelos en una lista:

In [3]:
df_append = pd.read_csv("./datos/ingesta_contaminacion.csv")

In [4]:
df_append = df_append.sort_values(['id_station','fecha'], ascending=[1, 1]).reset_index(drop=True)

In [8]:
df_append.head()

,CO,NO,NO2,NOX,O3,PM10,PM2.5,RH,SO2,TMP,WSP,dia,fecha,hora,id_station,mes
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,NaN,16.0,1.3,1,2019-08-01 01:00:00,1,ACO,8
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,NaN,15.0,1.9,1,2019-08-01 02:00:00,2,ACO,8
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.0,NaN,14.0,2.2,1,2019-08-01 03:00:00,3,ACO,8
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.0,NaN,13.0,1.9,1,2019-08-01 04:00:00,4,ACO,8
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.0,NaN,12.0,1.9,1,2019-08-01 05:00:00,5,ACO,8


In [9]:
df_append.head(5)

,CO,NO,NO2,NOX,O3,PM10,PM2.5,RH,SO2,TMP,WSP,dia,fecha,hora,id_station,mes
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,NaN,16.0,1.3,1,2019-08-01 01:00:00,1,ACO,8
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,NaN,15.0,1.9,1,2019-08-01 02:00:00,2,ACO,8
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.0,NaN,14.0,2.2,1,2019-08-01 03:00:00,3,ACO,8
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.0,NaN,13.0,1.9,1,2019-08-01 04:00:00,4,ACO,8
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.0,NaN,12.0,1.9,1,2019-08-01 05:00:00,5,ACO,8


## 3. 
### Generar columnas con información temporal como la hora, día y mes en que se generó la medición, estos atributos son de ultilidad para utilizar los patrones temporaloes de temporalidad en el fenómeno de contaminación.m

Arreglamos las fechas, vamos a generar columnas con el atributo hora,dia,mes.

In [11]:
df_append['dia'] = df_append['fecha'].astype(str).str[8:10]
df_append['mes'] =  df_append['fecha'].astype(str).str[5:7]
df_append['fecha'] = df_append['fecha'].astype(str).str[0:10]
df_append['dia'] = df_append['dia'].astype(str).astype(float)
df_append['mes'] = df_append['mes'].astype(str).astype(float)

Creamos una columna con toda esta información temporal, esta columna es de utilidad al ordenar y efectuar otras operaciones sobre el dataframe donde es necesario tener el orden de las mediciones bien definida.

In [13]:
df_append['fecha']  = pd.to_datetime([''.join([' '.join([df_append.loc[i, 'fecha'], 
                                                         str(df_append.loc[i, 'hora'])]),':00']) for i in df_append.index])
df_append['fecha'] = pd.to_datetime(df_append['fecha'], format='%Y-%m-%d %H:%M:%S')

In [14]:
df_append.head()

,CO,NO,NO2,NOX,O3,PM10,PM2.5,RH,SO2,TMP,WSP,dia,fecha,hora,id_station,mes
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,NaN,16.0,1.3,1.0,2019-08-01 01:00:00,1,ACO,8.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,NaN,15.0,1.9,1.0,2019-08-01 02:00:00,2,ACO,8.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.0,NaN,14.0,2.2,1.0,2019-08-01 03:00:00,3,ACO,8.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.0,NaN,13.0,1.9,1.0,2019-08-01 04:00:00,4,ACO,8.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.0,NaN,12.0,1.9,1.0,2019-08-01 05:00:00,5,ACO,8.0


In [15]:
df_append.head()

,CO,NO,NO2,NOX,O3,PM10,PM2.5,RH,SO2,TMP,WSP,dia,fecha,hora,id_station,mes
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,NaN,16.0,1.3,1.0,2019-08-01 01:00:00,1,ACO,8.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,NaN,15.0,1.9,1.0,2019-08-01 02:00:00,2,ACO,8.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.0,NaN,14.0,2.2,1.0,2019-08-01 03:00:00,3,ACO,8.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.0,NaN,13.0,1.9,1.0,2019-08-01 04:00:00,4,ACO,8.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.0,NaN,12.0,1.9,1.0,2019-08-01 05:00:00,5,ACO,8.0


In [16]:
df_append.head()

,CO,NO,NO2,NOX,O3,PM10,PM2.5,RH,SO2,TMP,WSP,dia,fecha,hora,id_station,mes
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,NaN,16.0,1.3,1.0,2019-08-01 01:00:00,1,ACO,8.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,NaN,15.0,1.9,1.0,2019-08-01 02:00:00,2,ACO,8.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.0,NaN,14.0,2.2,1.0,2019-08-01 03:00:00,3,ACO,8.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.0,NaN,13.0,1.9,1.0,2019-08-01 04:00:00,4,ACO,8.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.0,NaN,12.0,1.9,1.0,2019-08-01 05:00:00,5,ACO,8.0


Vamos a quedarnos con las celdas donde Pm10 y Pm2.5 son no nulas.

## 4. 
### Se filtran los valores de PM10 y PM2.5 que salen del percentil .85. 

Vemos que tenemos valores muy grandes, Jiang (2004) reportó que para valores mayores de 300 puntos, las mediciones no son confiables, vamos a quedarnos con mediciones menores a esta cota:

__Jiang, D., Zhang, Y., Hu, X., Zeng, Y., Tan, J., & Shao, D. (2004). Progress in developing an ANN model for air pollution index forecast. Atmospheric Environment, 38(40 SPEC.ISS.), 7055–7064. https://doi.org/10.1016/j.atmosenv.2003.10.066__

No queremos perder fechas, vamos a quedarnos con el valor anterior al outlier:

## 5. 
### Por último generamos el atributo PM10mean y PM25mean que representarán el promedio móvil de las últimas 24 horas de los dos contaminantes.


Según la [norma](http://www.aire.cdmx.gob.mx/default.php?ref=Z2Q=), la contingencia de PM10 y PM2.5 se considera cuando su promedio móvil de 24 horas excede 150 puntos, es necesario calcular el promedio móvil de cada contaminante para después poder pronosticarlo.

Volvemos a ordenar los datos de manera ascendente y ponemos la fecha cómo index:

In [17]:
df_append = df_append.sort_values(['fecha','id_station'], ascending=[1,1])

data_24 = df_append.set_index('fecha')

Ahora vamos a sacar el promedio de las ultimas 24 horas por estación para el PM10 y el PM25:

In [18]:
data_24 = data_24.groupby('id_station')[["PM10","PM2.5"]].rolling(24).mean().dropna().reset_index()

Renombramos estas columnas:m

In [19]:
data_24 = data_24.rename(columns={'PM10': 'PM10mean',"PM2.5":'PM25mean'})

##### Unimos el dataframe de los promedios móviles con el df original: 

In [20]:
data_day = pd.merge(df_append, data_24, on=["fecha","id_station"],how="left")

In [21]:
data_day.head(5)

,CO,NO,NO2,NOX,O3,PM10,PM2.5,RH,SO2,TMP,WSP,dia,fecha,hora,id_station,mes,PM10mean,PM25mean
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,NaN,16.0,1.3,1.0,2019-08-01 01:00:00,1,ACO,8.0,NaN,NaN
1,0.1,2.0,7.0,8.0,22.0,19.0,10.0,71.0,1.0,15.0,1.9,1.0,2019-08-01 01:00:00,1,AJM,8.0,NaN,NaN
2,NaN,NaN,NaN,NaN,23.0,NaN,5.0,NaN,NaN,8.0,1.1,1.0,2019-08-01 01:00:00,1,AJU,8.0,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2019-08-01 01:00:00,1,ARA,8.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2019-08-01 01:00:00,1,ATI,8.0,NaN,NaN


### Guardamos

In [22]:
data_day.to_csv(str("./datos/ingesta_contaminacion.csv"),index=False)